In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter

In [2]:
df=pd.read_csv("data.csv")

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,3,6,7,8,9,12,13,14,15,17,19,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['job_created_date'] =  pd.to_datetime(df['job_created_date'])
df['job_opening_date'] =  pd.to_datetime(df['job_opening_date'])
df['job_closing_date'] =  pd.to_datetime(df['job_closing_date'])

In [4]:
df=df[df.user_id.notnull()]

In [5]:
def get_all_users_id(df):
    users_id=df.user_id.values
    users_id=list(set(users_id))
    return users_id

In [6]:
all_users=get_all_users_id(df)

In [7]:
len(all_users)

165822

In [8]:
df=df[df.job_id.notnull()]

In [9]:
df.shape

(836373, 31)

In [10]:
df.tail(1)

,Unnamed: 0,user_id,job_created_date,job_application_status,job_id,employer_id,job_opening_date,job_closing_date,job_short_description,job_description,...,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,degree_subjectgroup_id,degree_subjectgroup_title
836372,671110,154122.0,2018-08-31 12:13:24,Initiated,2459.0,131.0,2018-08-17,2018-10-21,Greenhill & Co. is a leading independent inves...,<h2>The role</h2>\r\r\n\r\r\n<p>Genuine analys...,...,False,2019.0,True,751.0,345.0,University of Cambridge,2.0,Geography,4.0,Geography


In [11]:
df.head(1)

,Unnamed: 0,user_id,job_created_date,job_application_status,job_id,employer_id,job_opening_date,job_closing_date,job_short_description,job_description,...,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,degree_subjectgroup_id,degree_subjectgroup_title
0,0,75898.0,2017-09-01 01:00:02,Initiated,214.0,25.0,2017-03-27,2017-08-31,Think of our Summer Vacation Scheme in 2018 as...,<p>Apply now to fast track your application to...,...,False,2019.0,True,1371.0,381.0,University of Manchester,2.0,Spanish and Business,26.0,Modern Languages


In [12]:
def get_users_id(df):
    users_id=df.user_id.values
    users_id=list(set(users_id))
    return users_id

In [13]:
users_id=get_users_id(df)

In [14]:
len(users_id)

20812

In [15]:
def get_jobs_id(df):
    jobs_id=df.job_id.values
    jobs_id=list(set(jobs_id))
    return jobs_id

In [16]:
jobs_id=get_jobs_id(df)

In [17]:
len(jobs_id)

2988

In [18]:
def get_jobs_per_user(users_id, df):
    n=len(users_id)
    print("n :",n)
    print("users collected")
    jobs_per_user={}
    for i,user in enumerate(users_id):
        if i%1000==0:
            print("progress :", i/n)
        x=df[df["user_id"]==user]
        x=x[x.job_id.notnull()]
        if x.shape[0]>0:
            job_applications=x["job_id"].values
            job_applications=list(set(job_applications))
            jobs_per_user[str(int(user))]=job_applications
    print("Jobs per user collected")
    return jobs_per_user

In [19]:
jobs_per_user=get_jobs_per_user(users_id, df)

n : 20812
users collected
progress : 0.0
progress : 0.04804920238324044
progress : 0.09609840476648088
progress : 0.1441476071497213
progress : 0.19219680953296175
progress : 0.24024601191620218
progress : 0.2882952142994426
progress : 0.33634441668268306
progress : 0.3843936190659235
progress : 0.43244282144916396
progress : 0.48049202383240436
progress : 0.5285412262156448
progress : 0.5765904285988852
progress : 0.6246396309821257
progress : 0.6726888333653661
progress : 0.7207380357486066
progress : 0.768787238131847
progress : 0.8168364405150874
progress : 0.8648856428983279
progress : 0.9129348452815683
progress : 0.9609840476648087
Jobs per user collected


In [20]:
def get_jobs_to_recommend(all_users,df,jobs_per_user):
    n=len(all_users)
    recommendations_per_user={}
    now = datetime.datetime.now()
    #d=df[df['job_created_date']>=now-relativedelta(years=1)]
    c=df[df['job_closing_date'].isnull()]
    d=df[df['job_closing_date']>=now]
    d=d[d.job_id.notnull()]
    jobs_to_apply_to=list(d["job_id"].values)+list(c["job_id"].values)
    jobs_to_apply_to=list(set(jobs_to_apply_to))
    print("number of jobs to apply to :",len(jobs_to_apply_to))
    print("recommendations started")
    for i,user in enumerate(all_users):
        if i%10000==0:
            print("progress :", i/n)
        try:
            job_applications=jobs_per_user[str(int(user))]
            recommendations=list(set(jobs_to_apply_to) - set(job_applications))
            recommendations_per_user[str(int(user))]=recommendations
        except:
            recommendations_per_user[str(int(user))]=jobs_to_apply_to
    return recommendations_per_user

In [21]:
recommendations_per_user=get_jobs_to_recommend(all_users, df, jobs_per_user)

number of jobs to apply to : 294
recommendations started
progress : 0.0
progress : 0.060305628927404084
progress : 0.12061125785480817
progress : 0.18091688678221224
progress : 0.24122251570961634
progress : 0.30152814463702043
progress : 0.3618337735644245
progress : 0.4221394024918286
progress : 0.4824450314192327
progress : 0.5427506603466368
progress : 0.6030562892740409
progress : 0.663361918201445
progress : 0.723667547128849
progress : 0.783973176056253
progress : 0.8442788049836571
progress : 0.9045844339110612
progress : 0.9648900628384653


# create train and test dataset

In [22]:
a=list(jobs_per_user.keys())
train_user_id=[int(i) for i in a]
train_user_applications=list(jobs_per_user.values())

In [23]:
a=list(recommendations_per_user.keys())
test_user_id=[int(i) for i in a]
test_user_applications=list(recommendations_per_user.values())

In [24]:
class users:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.user_id=self.get_ids(d)
        self.length_of_course=self.get_length_of_course(d)
        self.gender=self.get_gender(d)
        self.ethnicity=self.get_ethnicity(d)
        self.school_type=self.get_school_type(d)
        self.course_includes_industrial_placement=self.get_course_includes_industrial_placement(d)
        self.graduation_year=self.get_graduation_year(d)
        self.right_to_work_uk=self.get_right_to_work_uk(d)
        self.degree_subject_id=self.get_degree_subject_id(d)
        self.university_id=self.get_university_id(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["user_id"]))
        return res
    
    def get_length_of_course(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["length_of_course"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_gender(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["gender"])
        return res
    
    def get_ethnicity(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["ethnicity"])
        return res
    
    def get_school_type(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["school_type"])
        return res
    
    def get_course_includes_industrial_placement(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["course_includes_industrial_placement"])
        return res
    
    def get_graduation_year(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["graduation_year"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_right_to_work_uk(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["right_to_work_uk"])
        return res
    
    def get_degree_subject_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["degree_subject_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_university_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["university_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["user_id"]=self.user_id
        dd["length_of_course"]=self.length_of_course
        dd["gender"]=self.gender
        dd["ethnicity"]=self.ethnicity
        dd["school_type"]=self.school_type
        dd["course_includes_industrial_placement"]=self.course_includes_industrial_placement
        dd["graduation_year"]=self.graduation_year
        dd["right_to_work_uk"]=self.right_to_work_uk
        dd["degree_subject_id"]=self.degree_subject_id
        dd["university_id"]=self.university_id
        self.df=pd.DataFrame(dd)

In [25]:
user=users("users_profile.json", "users_profile")
user_df=user.df

In [26]:
class career_preference:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.user_id=self.get_ids(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["profile_id"]))
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["jobsector_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["user_id"]=self.user_id
        dd["user_jobsectorid_preference"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [27]:
pref=career_preference("profile_sector_interests.json","profile_sector_interests")
career_preferences_df=pref.df

In [28]:
class jobs:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.job_id=self.get_ids(d)
        self.opening_date=self.get_opening_date(d)
        self.closing_date=self.get_closing_date(d)
        self.short_description=self.get_short_description(d)
        self.description=self.get_description(d)
        self.job_type=self.get_job_type(d)
        self.employer_id=self.get_employer_id(d)
        self.search_rank=self.get_search_rank(d)
        self.title=self.get_title(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_opening_date(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["opening_date"])
        return res
    
    def get_closing_date(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["closing_date"])
        return res
    
    def get_short_description(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description_short"])
        return res
    
    def get_description(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description"])
        return res
    
    def get_job_type(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["job_type"]))
        return res
    
    def get_employer_id(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["employer_id"]))
        return res
    
    def get_search_rank(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["search_rank"]))
        return res
    
    def get_title(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    def create_df(self):
        dd={}
        dd["job_id"]=self.job_id
        dd["job_opening_date"]=self.opening_date
        dd["job_closing_date"]=self.closing_date
        dd["job_short_description"]=self.short_description
        dd["job_description"]=self.description
        dd["job_type"]=self.job_type
        dd["employer_id"]=self.employer_id
        dd["search_rank"]=self.search_rank
        dd["job_title"]=self.title
        self.df=pd.DataFrame(dd)

In [29]:
job=jobs("job.json", "job")
job_df=job.df

In [30]:
class employers_job_sectors:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.employer_id=self.get_employer_id(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_employer_id(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["employer_id"]))
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["jobsector_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["employer_id"]=self.employer_id
        dd["employer_sector_id"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [31]:
employer_job_sector=employers_job_sectors("employers_sectors.json", "employers_sectors")
employer_job_sector_df=employer_job_sector.df

In [32]:
class jobsectors:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.title=self.get_title(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_title(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["jobsector_title"]=self.title
        dd["jobsector_id"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [33]:
jobsector=jobsectors("jobsectors.json", "job_sectors")
jobsector_df=jobsector.df

In [34]:
career_preferences_df=pd.merge(career_preferences_df, jobsector_df, left_on = 'user_jobsectorid_preference',
                               right_on = 'jobsector_id', how='outer')

In [35]:
career_preferences_df.drop("jobsector_id", axis=1, inplace=True)
career_preferences_df.drop("user_jobsectorid_preference", axis=1, inplace=True)

In [36]:
career_preferences_df=career_preferences_df.rename(index=str, columns={"jobsector_title": "user_preference"})

In [37]:
career_preferences_df.head()

,user_id,user_preference
0,35636,Commercial Law
1,35887,Commercial Law
2,35890,Commercial Law
3,35891,Commercial Law
4,35899,Commercial Law


In [38]:
class job_sectors:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.job_id=self.get_job_id(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_job_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["job_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["jobsector_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["job_id"]=self.job_id
        dd["jobsector_id"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [39]:
job_sector=job_sectors("job_sectors.json", "job_sectors")
job_sector_df=job_sector.df

In [40]:
class employers:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.employer_id=self.get_ids(d)
        self.title=self.get_title(d)
        self.description_short=self.get_description_short(d)
        self.description=self.get_description(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_title(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    def get_description_short(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description_short"])
        return res
    
    
    def get_description(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description"])
        return res
    
    
    def create_df(self):
        dd={}
        dd["employer_id"]=self.employer_id
        dd["employer_title"]=self.title
        dd["employer_description_short"]=self.description_short
        dd["employer_description"]=self.description
        self.df=pd.DataFrame(dd)

In [41]:
employer=employers("employer.json", "employer")
employer_df=employer.df

In [42]:
class university:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.university_id=self.get_ids(d)
        self.university_name=self.get_university_name(d)
        self.university_type=self.get_university_type(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_university_name(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["name"])
        return res
    
    def get_university_type(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["university_type"]))
        return res
    
    def create_df(self):
        dd={}
        dd["university_id"]=self.university_id
        dd["university_name"]=self.university_name
        dd["university_type"]=self.university_type
        self.df=pd.DataFrame(dd)

In [43]:
uni=university("university.json", "university")
uni_df=uni.df

In [44]:
class subject:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.subject_id=self.get_ids(d)
        self.subject_name=self.get_subject_name(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_subject_name(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["name"])
        return res
    
    
    def create_df(self):
        dd={}
        dd["degree_subject_id"]=self.subject_id
        dd["degree_subject_name"]=self.subject_name
        self.df=pd.DataFrame(dd)

In [45]:
sub=subject("subject.json", "subject")
subject_df=sub.df

In [46]:
class subject_groups:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.subject_id=self.get_ids(d)
        self.subjectgroup_id=self.get_subjectgroup_id(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["subject_id"]))
        return res
    
    def get_subjectgroup_id(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["subjectgroup_id"]))
        return res
    
    
    def create_df(self):
        dd={}
        dd["degree_subject_id"]=self.subject_id
        dd["degree_subjectgroup_id"]=self.subjectgroup_id
        self.df=pd.DataFrame(dd)

In [47]:
subgroup=subject_groups("subject_groups.json", "subject_groups")
subjectgroup_df=subgroup.df

In [48]:
class subjectgroup:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.subjectgroup_id=self.get_ids(d)
        self.subjectgroup_title=self.get_subject_name(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_subject_name(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    
    def create_df(self):
        dd={}
        dd["degree_subjectgroup_id"]=self.subjectgroup_id
        dd["degree_subjectgroup_title"]=self.subjectgroup_title
        self.df=pd.DataFrame(dd)

In [49]:
subgrouptitle=subjectgroup("subjectgroup.json", "subjectgroup")
subjectgrouptitle_df=subgrouptitle.df

In [50]:
job_df=pd.merge(job_df, employer_df, on='employer_id', how='outer')

In [51]:
job_df=pd.merge(job_df, employer_job_sector_df, on='employer_id', how='outer')

In [52]:
job_df=pd.merge(job_df, jobsector_df, left_on = 'employer_sector_id', right_on = 'jobsector_id', how='outer')

In [53]:
job_df=job_df.rename(index=str, columns={"jobsector_title": "employer_sector_title"})

In [54]:
job_df.drop("jobsector_id", axis=1, inplace=True)

In [55]:
job_df=pd.merge(job_df, job_sector_df, on='job_id', how='outer')

In [56]:
job_df=pd.merge(job_df, jobsector_df, on = 'jobsector_id', how='outer')

In [57]:
job_df=job_df.rename(index=str, columns={"jobsector_title": "job_sector_title"})

In [58]:
user_df=pd.merge(user_df, uni_df, on='university_id', how='outer')

In [59]:
user_df=pd.merge(user_df, subject_df, on='degree_subject_id', how='outer')

In [60]:
user_df=pd.merge(user_df, subjectgroup_df, on='degree_subject_id', how='outer')

In [61]:
#user_df=pd.merge(user_df, subjectgrouptitle_df, on='degree_subjectgroup_id', how='outer')

In [62]:
user_df.drop("degree_subjectgroup_id", axis=1, inplace=True)

In [63]:
user_df=pd.merge(user_df, career_preferences_df, on='user_id', how='outer')

In [64]:
user_df=user_df.drop_duplicates(subset=user_df.columns)

In [65]:
user_df[user_df["user_preference"].notnull()]

,user_id,length_of_course,gender,ethnicity,school_type,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,user_preference
0,146656.0,NaN,Female,Mixed/Multiple Ethnic Background,None,false,2019.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Investment Banking
1,146656.0,NaN,Female,Mixed/Multiple Ethnic Background,None,false,2019.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Investment & Asset Management
2,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Energy & Infrastructure
3,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Investment Banking
4,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Investment & Asset Management
5,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Financial Services
6,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Consulting
7,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Entrepreneurship & Start Ups
8,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Public Sector & Government
9,192546.0,4.0,Female,White/White British,Grammar,true,2021.0,true,195.0,333.0,University of Birmingham,2.0,Business and Management,Charity & Social Enterprise


In [66]:
#job_df=job_df[job_df["employer_sector_title"]!=job_df["job_sector_title"]]

In [67]:
job_df[job_df["job_id"]==3163.0]["job_opening_date"]

7871    2017-10-01
8827    2017-10-01
Name: job_opening_date, dtype: object

In [68]:
job_df.columns

Index(['job_id', 'job_opening_date', 'job_closing_date',
       'job_short_description', 'job_description', 'job_type', 'employer_id',
       'search_rank', 'job_title', 'employer_title',
       'employer_description_short', 'employer_description',
       'employer_sector_id', 'employer_sector_title', 'jobsector_id',
       'job_sector_title'],
      dtype='object')

In [69]:
user_df.shape

(1170903, 14)

In [70]:
job_df.shape

(14348, 16)

In [71]:
def make_dataset(job_df, user_df, ids, applications):
    n=len(ids)
    d={}
    user_id=[]
    job_id=[]
    employer_id=[]
    job_opening_date=[]
    job_closing_date=[]
    job_description=[]
    job_type=[]
    job_title=[]
    employer_title=[]
    employer_description=[]
    employer_sector_id=[]
    employer_sector_title=[]
    jobsector_id=[]
    job_sector_title=[]
    gender=[]
    course_includes_industrial_placement=[]
    graduation_year=[]
    right_to_work_uk=[]
    degree_subject_id=[]
    university_id=[]
    university_name=[]
    university_type=[]
    degree_subject_name=[]
    career_preference=[]
    for i in range(n):
        if i%1000==0:
            print("progress :",i/n)
        user_applications=applications[i]
        nb_applications=len(user_applications)
        for j in range(nb_applications):
            user_id.append(ids[i])
            job_id.append(user_applications[j])
            user_info=user_df[user_df["user_id"]==ids[i]]
            job_infos=job_df[job_df["job_id"]==user_applications[j]]
            employer_id.append(job_infos["employer_id"].iloc[0])
            job_opening_date.append(job_infos["job_opening_date"].iloc[0])
            job_closing_date.append(job_infos["job_closing_date"].iloc[0])
            job_description.append(job_infos["job_description"].iloc[0])
            job_type.append(job_infos["job_type"].iloc[0])
            job_title.append(job_infos["job_title"].iloc[0])
            employer_title.append(job_infos["employer_title"].iloc[0])
            employer_description.append(job_infos["employer_description"].iloc[0])
            employer_sector_id.append(job_infos["employer_sector_id"].iloc[0])
            employer_sector_title.append(job_infos["employer_sector_title"].iloc[0])
            jobsector_id.append(job_infos["jobsector_id"].iloc[0])
            job_sector_title.append(job_infos["job_sector_title"].iloc[0])
            try:
                gender.append(user_info["gender"].iloc[0])
            except:
                gender.append(None)
            try:
                course_includes_industrial_placement.append(user_info["course_includes_industrial_placement"].iloc[0])
            except:
                course_includes_industrial_placement.append(None)
            try:
                graduation_year.append(user_info["graduation_year"].iloc[0])
            except:
                graduation_year.append(None)
            try:
                right_to_work_uk.append(user_info["right_to_work_uk"].iloc[0])
            except:
                right_to_work_uk.append(None)
            try:
                degree_subject_id.append(user_info["degree_subject_id"].iloc[0])
            except:
                degree_subject_id.append(None)
            try:
                university_id.append(user_info["university_id"].iloc[0])
            except:
                university_id.append(None)
            try:
                university_name.append(user_info["university_name"].iloc[0])
            except:
                university_name.append(None)
            try:
                university_type.append(user_info["university_type"].iloc[0])
            except:
                university_type.append(None)
            try:
                degree_subject_name.append(user_info["degree_subject_name"].iloc[0])
            except:
                degree_subject_name.append(None)
            try:
                career_preference.append(user_info["user_preference"].iloc[0])
            except:
                career_preference.append(None)
    d["user_id"]=user_id
    d["job_id"]=job_id
    d["employer_id"]=employer_id
    d["job_opening_date"]=job_opening_date
    d["job_closing_date"]=job_closing_date
    d["job_description"]=job_description
    d["job_type"]=job_type
    d["job_title"]=job_title
    d["employer_title"]=employer_title
    d["employer_description"]=employer_description
    d["employer_sector_id"]=employer_sector_id
    d["employer_sector_title"]=employer_sector_title
    d["jobsector_id"]=jobsector_id
    d["job_sector_title"]=job_sector_title
    d["gender"]=gender
    d["course_includes_industrial_placement"]=course_includes_industrial_placement
    d["graduation_year"]=graduation_year
    d["right_to_work_uk"]=right_to_work_uk
    d["degree_subject_id"]=degree_subject_id
    d["university_id"]=university_id
    d["university_name"]=university_name
    d["university_type"]=university_type
    d["degree_subject_name"]=degree_subject_name
    d["user_preference"]=career_preference
    df=pd.DataFrame(d)
    return df

In [72]:
training_set=make_dataset(job_df, user_df, train_user_id, train_user_applications)

progress : 0.0
progress : 0.04804920238324044
progress : 0.09609840476648088
progress : 0.1441476071497213
progress : 0.19219680953296175
progress : 0.24024601191620218
progress : 0.2882952142994426
progress : 0.33634441668268306
progress : 0.3843936190659235
progress : 0.43244282144916396
progress : 0.48049202383240436
progress : 0.5285412262156448
progress : 0.5765904285988852
progress : 0.6246396309821257
progress : 0.6726888333653661
progress : 0.7207380357486066
progress : 0.768787238131847
progress : 0.8168364405150874
progress : 0.8648856428983279
progress : 0.9129348452815683
progress : 0.9609840476648087


In [73]:
training_set.shape

(125136, 24)

In [74]:
training_set.columns

Index(['user_id', 'job_id', 'employer_id', 'job_opening_date',
       'job_closing_date', 'job_description', 'job_type', 'job_title',
       'employer_title', 'employer_description', 'employer_sector_id',
       'employer_sector_title', 'jobsector_id', 'job_sector_title', 'gender',
       'course_includes_industrial_placement', 'graduation_year',
       'right_to_work_uk', 'degree_subject_id', 'university_id',
       'university_name', 'university_type', 'degree_subject_name',
       'user_preference'],
      dtype='object')

In [78]:
training_set[training_set["user_preference"].notnull()].sample(5)

,user_id,job_id,employer_id,job_opening_date,job_closing_date,job_description,job_type,job_title,employer_title,employer_description,...,gender,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,user_preference
116677,127826,2535.0,95.0,2018-09-01,2018-10-31,<h2>Overview</h2>\r\n\r\n<p>Alpha Financial Ma...,1.0,Alpha Financial Markets Consulting Analyst,Alpha FMC,<h3>Alpha Financial Markets Consulting</h3>\r\...,...,Male,false,2017.0,true,462.0,375.0,University of Kent,2.0,Economics,Consulting
28823,108933,4165.0,76.0,None,2018-02-28,<h2>Overview</h2>\r\n\r\n<p>Structured Finance...,3.0,Customer Solutions - Structured Finance Intern...,RBS,"<p>The passion you own, the expertise you deve...",...,Male,false,2019.0,true,1282.0,392.0,University of Oxford,2.0,Ppe,Commercial Law
115761,127682,213.0,32.0,2017-09-25,None,<h2>What we do</h2>\r\n\r\n<p>Our purpose is t...,3.0,Women in Business (final years) - 2018,PwC,<p>Opportunities are at the heart of a career ...,...,Female,false,2019.0,true,401.0,375.0,University of Kent,2.0,Criminology and Sociology,Commercial Law
7295,166200,880.0,227.0,2017-09-13,2017-12-31,<h2>What you&rsquo;ll do</h2>\r\n\r\n<p>Over t...,1.0,Analytical Marketing Programme,E.ON,<p>E.ON UK provides electricity and gas to mil...,...,Male,false,2020.0,true,357.0,287.0,Queen Mary University of London,2.0,Computer Science with Management,"Consumer, FMCG & Retail"
48044,114984,519.0,357.0,2017-04-05,None,<h2>Overview</h2>\r\n\r\n<p>The Worthwhile Gra...,1.0,Graduate Scheme,Worthwhile,<p>Worthwhile creates career paths for young p...,...,Female,false,2018.0,true,947.0,333.0,University of Birmingham,2.0,International Relations with Political Science,Commercial Law


In [83]:
test=training_set[training_set["user_id"]==114984][['job_title',
       'employer_title', 
       'employer_sector_title', 'job_sector_title', 'gender',
       'graduation_year','university_name','degree_subject_name',"user_preference"]]

In [84]:
test.tail(10)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,graduation_year,university_name,degree_subject_name,user_preference
48054,Graduate Programme,Insight Investment,Investment & Asset Management,Financial Services,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48055,Communications Manager,TPP,Technology: IT & Software Development,Technology: IT & Software Development,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48056,Training Contract 2020,Taylor Wessing,Commercial Law,Commercial Law,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48057,Sales/Business Development Graduate,Fujitsu,Technology: Consulting & Project Management,Technology: Consulting & Project Management,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48058,"Global CEO Challenge 2018 ? United Kingdom, Ir...",P&G,"Consumer, FMCG & Retail",Financial Services,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48059,Tax Professionals Graduate Programme,HM Revenue & Customs,Public Sector & Government,Public Sector & Government,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48060,UK Sales Graduate Job 2019,P&G,"Consumer, FMCG & Retail",Sales & Commercial,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48061,Training Contract 2020,Kennedys,Commercial Law,Commercial Law,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48062,Transformation Graduate Leadership Programme,Lloyds Banking Group,Financial Services,Technology: Consulting & Project Management,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law
48063,Graduate Programme - Area Manager,Aldi,"Consumer, FMCG & Retail",Management & Business,Female,2018.0,University of Birmingham,International Relations with Political Science,Commercial Law


In [82]:
test_set=make_dataset(job_df, user_df, test_user_id[1000:1050], test_user_applications[1000:1050])

progress : 0.0


In [85]:
test=test_set[test_set["user_preference"].notnull()][['job_title',
       'employer_title', 
       'employer_sector_title', 'job_sector_title', 'gender',
       'graduation_year','university_name','degree_subject_name',"user_preference"]]

In [86]:
test.head()

,job_title,employer_title,employer_sector_title,job_sector_title,gender,graduation_year,university_name,degree_subject_name,user_preference


In [87]:
training_set.to_csv("Training_set.csv")
test_set.to_csv("Test_set.csv")